## Home Prices Report

This notebook analyzes home price data.

In [1]:
import datetime
from dateutil.relativedelta import relativedelta
from pyfredapi import FredSeries
from rich import print as rprint
import pandas as pd
import plotly.express as px
from utils.state_abb import state_abb
from utils.pandas_utils import get_dates, calc_pct_chg, display_pct_chg_df

In [2]:
client = FredSeries()

In [3]:
state_home_price_index_series = [f"{abb}STHPI" for abb in state_abb.values()]

case_shiller_series = [
    "CSUSHPISA",
    "SPCS20RSA",
    "SFXRSA",
    "LXXRSA",
    "SDXRSA",
    "NYXRSA",
    "CHXRSA",
    "SEXRSA",
    "BOXRSA",
    "PHXRSA",
    "MIXRSA",
    "DNXRSA",
    "DAXRSA",
    "WDXRSA",
    "ATXRSA",
    "LVXRSA",
    "POXRSA",
    "TPXRSA",
    "CRXRSA",
    "MNXRSA",
    "DEXRSA",
    "CEXRSA",
]

housing_starts_series = ["HOUST", "HOUST1F", "HOUST2F", "HOUST5F"]
housing_sale_price_series = ["MSPUS", "MSPNHSUS"]
house_supply_series = ["MSACSRNSA"]
mortgage_rate_series = ["MORTGAGE30US", "MORTGAGE15US"]
bond_yield_series = ["WGS10YR", "WGS20YR", "WGS30YR"]
fed_mbs_series = ["WSHOMCB"]
home_ownership_rates_series = ["RHORUSQ156N"]
median_hh_income_series = ["MEHOINUSA672N"]
mortgage_debt_as_perc_of_income_series = "MDSP"

# Calculate the spread on the 30-year mortgage and the 10-year yield

In [4]:
home_sale_price_response = {
    s: client.get_series(s)
    for s in housing_starts_series
}

In [10]:
home_sale_price_response["HOUST"].info.units

'Thousands of Units'

In [9]:
home_sale_price_response["HOUST"].data

,realtime_start,realtime_end,date,value
0,2022-10-13,2022-10-13,1959-01-01,1657.0
1,2022-10-13,2022-10-13,1959-02-01,1667.0
2,2022-10-13,2022-10-13,1959-03-01,1620.0
3,2022-10-13,2022-10-13,1959-04-01,1590.0
4,2022-10-13,2022-10-13,1959-05-01,1498.0
...,...,...,...,...
759,2022-10-13,2022-10-13,2022-04-01,1805.0
760,2022-10-13,2022-10-13,2022-05-01,1562.0
761,2022-10-13,2022-10-13,2022-06-01,1575.0
762,2022-10-13,2022-10-13,2022-07-01,1404.0


In [11]:
housing_sale_price_series_response = {
    s: client.get_series(s)
    for s in housing_sale_price_series
}

In [20]:
housing_sale_price_series_response["MSPUS"].info.title

'Median Sales Price of Houses Sold for the United States'

In [21]:
mortgage_rate_series_responses = {
    s: client.get_series(s)
    for s in mortgage_rate_series
}

In [24]:
mortgage_rate_series_responses["MORTGAGE30US"].data.dtypes

realtime_start            object
realtime_end              object
date              datetime64[ns]
value                    float64
dtype: object

## Collect home price data

# Median Sale Price

# Housing Starts

In [ ]:
housing_starts_response = {s: client.get_series(s) for s in housing_starts_series}

# State House Price Index

In [ ]:
state_hp_responses = {s: client.get_series(s) for s in state_home_price_index_series}

### Munge the data

In [ ]:
state_hp_dfs = []
state_hp_wide_dfs = []

for s in state_home_price_index_series:
    state_name = state_hp_responses[s].info.title.replace(
        "All-Transactions House Price Index for ", ""
    )

    df = (
        state_hp_responses[s]
        .data.drop(["realtime_start", "realtime_end"], axis=1)
        .copy()
    )
    df["state"] = state_name
    state_hp_dfs.append(df)

    wide_df = (
        state_hp_responses[s]
        .data.drop(["realtime_start", "realtime_end"], axis=1)
        .copy()
    )
    wide_df = wide_df.rename(columns={"value": state_name}).set_index("date")
    state_hp_wide_dfs.append(wide_df)

state_hp_df = pd.concat(state_hp_dfs, axis=0)
state_hp_wide_df = pd.concat(state_hp_wide_dfs, axis=1)

In [ ]:
state_hp_dates = get_dates(state_hp_df, "date")
rprint(f"Minimum home price date: {state_hp_dates.min}")
rprint(f"Maximum home price date: {state_hp_dates.max}")

### Percent Change Table

In [ ]:
state_hp_pct_chg_df = calc_pct_chg(
    wide_df=state_hp_wide_df.copy(),
    lags=[1, 4, 8],
    max_date=state_hp_dates.max,
    grp_var="State",
    lag_var="Quarters Ago",
    title="State Home Price Index Percent Change from Preceding Quarter",
)

# rprint(f"State Home Price Index Percent Change from Preceding Quarter, {state_hp_dates.max}")
display(state_hp_pct_chg_df)

### Plot

In [ ]:
line_plot = px.line(
    data_frame=state_hp_df,
    x="date",
    y="value",
    color="state",
    title=f"Quarterly Home Price Index by State (Index 1980:Q1=100)",
    labels=dict(value="Home Price Index", date="Date", state="State"),
)

line_plot.show(renderer="notebook")

# Case/Shiller Home Price Index

In [ ]:
cs_responses = {s: client.get_series(s) for s in case_shiller_series}

### Munge the data

In [ ]:
cs_dfs = []
cs_wide_dfs = []

for s in case_shiller_series:
    series_name = (
        cs_responses[s]
        .info.title.replace("S&P/Case-Shiller ", "")
        .replace(" Home Price Index", "")
    )

    df = cs_responses[s].data.drop(["realtime_start", "realtime_end"], axis=1).copy()

    df["series"] = series_name
    cs_dfs.append(df)

    wide_df = (
        cs_responses[s].data.drop(["realtime_start", "realtime_end"], axis=1).copy()
    )
    wide_df = wide_df.rename(columns={"value": series_name}).set_index("date")
    cs_wide_dfs.append(wide_df)

cs_df = pd.concat(cs_dfs)
cs_wide_df = pd.concat(cs_wide_dfs, axis=1)

In [ ]:
cs_dates = get_dates(cs_df, "date")
rprint(f"Minimum home price date: {cs_dates.min}")
rprint(f"Maximum home price date: {cs_dates.max}")

### Percent Change Table

In [ ]:
cs_pct_chg_df = calc_pct_chg(
    wide_df=cs_wide_df.copy().dropna(),
    lags=[1, 3, 6, 12, 18, 24, 36, 48, 60],
    max_date=cs_dates.max,
    grp_var="Series",
    lag_var="Months Ago",
    title="Case/Shiller Index Percent Change from Preceding Quarter",
)

display_pct_chg_df(
    df=cs_pct_chg_df.copy(),
    title="Seasonally Adjusted CPI, Percent Change from Preceding Month",
    max_date=dates.max,
)

display(cs_pct_chg_df)

### Plot

In [ ]:
line_plot = px.line(
    data_frame=cs_df.dropna(),
    x="date",
    y="value",
    color="series",
    title=f"Case/Shiller Home Price Index (Index Jan-2000=100)",
    labels=dict(value="CS Home Price Index", date="Date", series="Series"),
)

line_plot.show(renderer="notebook")